In [ ]:
import polars as pl
import numpy as np
import implicit
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from scipy.sparse import coo_matrix, csr_array
from amazon.models import normalize_scores  
import random
import gc
random.seed(42)

In [2]:
lf: pl.LazyFrame = pl.scan_parquet("data/processed/amazon-2023.parquet")
lf = lf.select(["user_id", "asin", "rating"])
lf = lf.filter(pl.len().over("user_id") >= 5)

lf = lf.with_columns([
    pl.col("user_id").cast(pl.Categorical).alias("user_cat"),
    pl.col("asin").cast(pl.Categorical).alias("item_cat")
]).drop(["user_id", "asin"])

lf: pl.LazyFrame = lf.with_columns([
    pl.col("user_cat").to_physical().alias("user_idx"),
    pl.col("item_cat").to_physical().alias("item_idx")
])

df: pl.DataFrame = lf.collect()

In [3]:
unique_users, unique_items = df["user_cat"].unique().sort(), df["item_cat"].unique().sort()

user_id_map = {i: user_str for i, user_str in enumerate(unique_users.to_list())}
item_id_map = {i: item_str for i, item_str in enumerate(unique_items.to_list())}

ratings_np = df['rating'].cast(pl.Float32).to_numpy()
user_indices_np = df['user_idx'].to_numpy()
item_indices_np = df['item_idx'].to_numpy()

num_users, num_items = len(user_id_map), len(item_id_map)
train_indices, test_indices = train_test_split(np.arange(len(df)), test_size=0.2, random_state=42)

train_user_indices, train_item_indices = user_indices_np[train_indices], item_indices_np[train_indices]
train_ratings = ratings_np[train_indices]

test_user_indices, test_item_indices = user_indices_np[test_indices], item_indices_np[test_indices]
test_ratings = ratings_np[test_indices]

del df
gc.collect()

20

In [4]:
train_sparse_csr: csr_array = coo_matrix((train_ratings, (train_user_indices, train_item_indices)),
                            shape=(num_users, num_items)).tocsr()

In [5]:
model = implicit.als.AlternatingLeastSquares(
    factors=50, regularization=0.01, iterations=15, random_state=42, use_gpu=False
)
model.fit(train_sparse_csr)

/home/glor/300/comp3610A3/venv/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 24 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 15/15 [1:11:14<00:00, 284.96s/it]


In [ ]:
predicted_ratings_test: list[float] = [
    model.user_factors[user_idx, :].dot(model.item_factors[item_idx, :])
    for user_idx, item_idx, actual in zip(test_user_indices, test_item_indices, test_ratings)
    if user_idx < model.user_factors.shape[0] and item_idx < model.item_factors.shape[0]
]

if predicted_ratings_test:
    rmse: float = root_mean_squared_error(test_ratings[:len(predicted_ratings_test)], predicted_ratings_test)
    print(f"Root-mean-square error (RMSE) on test data = {rmse:.4f}")
else:
    print("Warning: No test ratings could be predicted.")
    rmse = float('nan')

# Sample Root-mean-square error (RMSE) on test data = 4.4926
# Full Root-mean-square error (RMSE) on test data = 4.4181

Root-mean-square error (RMSE) on test data = 4.4181


In [7]:
# %% Demo: Top‑5 recommendations for 3 random test users
unique_test_users: np.ndarray = np.unique(test_user_indices)
demo_users: list[int] = random.sample(list(unique_test_users), 3)

for user_idx in demo_users:
    scores = model.user_factors[user_idx].dot(model.item_factors.T)
    seen = set(train_item_indices[train_user_indices == user_idx])
    scores[list(seen)] = -np.inf  # mask seen

    scores: np.ndarray = normalize_scores(scores)
    top5_idx: np.ndarray = np.argpartition(scores, -5)[-5:]
    top5_idx = top5_idx[np.argsort(-scores[top5_idx])]

    print(f"\nUser {user_id_map[user_idx]} (idx={user_idx}):")
    for item_idx in top5_idx:
        print(f"  ASIN {item_id_map[item_idx]} — pred. rating {scores[item_idx]:.2f}")


User AECVLWXQSOTOA4R7IB4KWTUB4H3Q (idx=3970955):
  ASIN B01MFGX5GI — pred. rating 5.00
  ASIN B07Q5TL9SQ — pred. rating 4.78
  ASIN B06Y1264PX — pred. rating 4.76
  ASIN B00K2EOONI — pred. rating 4.73
  ASIN B07MV8SWZF — pred. rating 4.68

User AFTKJPWKJIC7R3J23RVC5JYPUJGA (idx=876054):
  ASIN B00DS842HS — pred. rating 5.00
  ASIN B00FLYWNYQ — pred. rating 4.44
  ASIN B0009X29WK — pred. rating 4.36
  ASIN B00016XJ4M — pred. rating 4.16
  ASIN B0026HDURA — pred. rating 4.14

User AEJPJARKMDB6YBVFJDWYGGONXV4Q (idx=10046557):
  ASIN B079QHML21 — pred. rating 5.00
  ASIN B001T7QJ9O — pred. rating 4.50
  ASIN B0043T7FXE — pred. rating 4.49
  ASIN B004S8F7QM — pred. rating 4.37
  ASIN B003NR57BY — pred. rating 4.12
